<a href="https://colab.research.google.com/github/akiphumi/todoist/blob/master/todoist_random_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* [Gets and prints the tasks for "today", from the Todoist API. Intended to be used alongside https://github.com/Doist/todoist-python.](https://gist.github.com/fpapado/9e1e6bd133664376eaa5f6855712f28b)
* [API Documentation | Todoist Developer](https://developer.todoist.com/sync/v8/?python#)
* [todoist-python/test_api.py at master · Doist/todoist-python](https://github.com/Doist/todoist-python/blob/master/tests/test_api.py)

### ライブラリのインストール

In [0]:
! pip install todoist-python

### ライブラリのインポートと同期
[HTTP request limits · Issue #18 · Garee/pytodoist](https://github.com/Garee/pytodoist/issues/18)

In [0]:
import random
import time
from todoist.api import TodoistAPI
from tqdm import tqdm

# API token は Settting の Interations で確認
token = '0123456789abcdef0123456789abcdef01234567'

api = TodoistAPI(token)
api.sync()

## 1分間に50回の同期に制限
[HTTP request limits · Issue #18 · Garee/pytodoist](https://github.com/Garee/pytodoist/issues/18)

In [0]:
# Add to the top of your code before you start using the API.

# Decorator to limit a function call to X per second.
# Credit: http://blog.gregburek.com/2011/12/05/Rate-limiting-with-decorators/
def rate_limit(maxPerSecond):
    minInterval = 1.0 / float(maxPerSecond)
    
    def decorate(func):
        lastTimeCalled = [0.0]
        
        def rateLimitedFunction(*args, **kargs):
            elapsed = time.clock() - lastTimeCalled[0]
            leftToWait = minInterval - elapsed
            if leftToWait > 0:
                time.sleep(leftToWait)
            ret = func(*args, **kargs)
            lastTimeCalled[0] = time.clock()
            return ret
 
        return rateLimitedFunction
    return decorate

# The normal sync function
commit = api.commit

# The rate limited version of the sync function
@rate_limit(0.8)  # 50/60 = 0.833...
def rate_limited_commit(*args):
#     print("Syncing...")
    return commit(*args)

# Use the rate limited sync function
api.commit = rate_limited_commit

## ラベルの辞書を作って

In [0]:
labels = {}
for label in api.state['labels']:
  labels.setdefault(label['name'], label['id'])
print(labels)

## アイテムのリストを作って

In [0]:
items = []
# 同期したいラベルを指定
key = 'next'
for item in api.state['items']:
  if labels.get(key) in item['labels']:
    items.append(item['id'])
print(items)

## Daily Goalの数だけランダムに選択

In [0]:
daily_goals = random.sample(items, api.state['user']['daily_goal'])
print(daily_goals)

## 実行

In [0]:
for item in tqdm(api.state['items']):
  if item['id'] in daily_goals:
    item.update(due={'string': 'today'})
    api.commit()